In [2]:
from risf.distance import *
from risf.distance_functions import *
import pandas as pd
import numpy as np
import pickle

In [3]:
og_columns = ['WetTankAirPressure', 'LongitudAcc', 'EngineSpeed', 'Fuel Rate',
       'Engine Load', 'Boost Pressure', 'EngineAirInletPressure',
       'AcceleratorPedalPos', 'VehicleSpeed', 'BrakePedalPos']

In [4]:
df_369 = pd.read_parquet('../data/bus/REP_369.parquet')
df_370 = pd.read_parquet('../data/bus/REP_370.parquet')
df_371 = pd.read_parquet('../data/bus/REP_371.parquet')
df_372 = pd.read_parquet('../data/bus/REP_372.parquet')
df_375 = pd.read_parquet('../data/bus/REP_375.parquet')

In [5]:
def counts_to_distribution(counts):
    total_count = np.sum(counts)
    distribution = counts / total_count
    return distribution

In [6]:
for x in [df_369, df_370, df_371, df_372, df_375]:
    selected_columns = x.filter(regex='HistDens*')
    x.drop(selected_columns.columns, axis=1, inplace=True)

In [7]:
vals_columns = df_369.filter(regex='HistVals*')
dist_columns = vals_columns.applymap(counts_to_distribution)
dist_columns.columns = dist_columns.columns.str.replace('HistVals', 'HistDens')
df_369 = pd.concat([df_369, dist_columns], axis=1)

vals_columns = df_370.filter(regex='HistVals*')
dist_columns = vals_columns.applymap(counts_to_distribution)
dist_columns.columns = dist_columns.columns.str.replace('HistVals', 'HistDens')
df_370 = pd.concat([df_370, dist_columns], axis=1)

vals_columns = df_371.filter(regex='HistVals*')
dist_columns = vals_columns.applymap(counts_to_distribution)
dist_columns.columns = dist_columns.columns.str.replace('HistVals', 'HistDens')
df_371 = pd.concat([df_371, dist_columns], axis=1)

vals_columns = df_372.filter(regex='HistVals*')
dist_columns = vals_columns.applymap(counts_to_distribution)
dist_columns.columns = dist_columns.columns.str.replace('HistVals', 'HistDens')
df_372 = pd.concat([df_372, dist_columns], axis=1)

vals_columns = df_375.filter(regex='HistVals*')
dist_columns = vals_columns.applymap(counts_to_distribution)
dist_columns.columns = dist_columns.columns.str.replace('HistVals', 'HistDens')
df_375 = pd.concat([df_375, dist_columns], axis=1)

In [8]:
X_train = pd.concat([df_370, df_371, df_372, df_375])
X_test = df_369
# X_train.to_parquet('CS_train.parquet')
# X_test.to_parquet('CS_test.parquet')

In [ ]:
DISTANCES_PATH = '../data/bus/distances/'

In [59]:
numeric_reps = ['mean', 'std', 'skewness', 'kurtosis', 'quan_0', 'quan_25', 'quan_50', 'quan_75', 'quan_100',
                'dec_10', 'dec_20', 'dec_30', 'dec_40', 'dec_50', 'dec_60', 'dec_70', 'dec_80', 'dec_90']

In [56]:
dff = X_train['Numeric_WetTankAirPressure'].values[:800]
dff = np.vstack(dff)
dff[:,0].reshape(-1,1)

array([[ 6.54262149],
       [ 6.95329678],
       [ 6.92801419],
       [ 6.78209437],
       [ 6.64132146],
       [ 6.53646   ],
       [ 6.44653771],
       [ 6.40602958],
       [ 6.36955345],
       [ 0.57640741],
       [ 1.01402467],
       [ 1.17000489],
       [ 1.4427635 ],
       [ 1.71921808],
       [ 1.90095483],
       [ 1.99856029],
       [ 2.07460495],
       [ 2.13745   ],
       [ 2.15259509],
       [ 4.66504565],
       [ 8.63802174],
       [ 8.46382531],
       [ 8.29726024],
       [ 8.13231616],
       [ 7.98301458],
       [ 7.00659538],
       [ 5.08247688],
       [ 5.23925548],
       [10.80856709],
       [10.55048033],
       [ 5.08834583],
       [10.48461361],
       [11.46503963],
       [11.64335667],
       [11.19384671],
       [10.80148669],
       [10.39953272],
       [10.07098261],
       [11.98410586],
       [11.51254787],
       [11.10136042],
       [10.69733041],
       [10.29706474],
       [10.92606773],
       [11.83011986],
       [11

In [ ]:
def dist_4_numeric(X_train, X_test, column_name, base_path, n_jobs):
    PATHS = []
    X_train = np.vstack(X_train)
    X_test = np.vstack(X_test)
    for i, rep in enumerate(numeric_reps):
        
        path = base_path + column_name.replace(' ', '_') + '_' + numeric_reps[i]
        TRAIN_DIST_PATH = path + '_euq_tr.pickle'
        TEST_DIST_PATH = path + '_euq_te.pickle'
        PATHS.append(TRAIN_DIST_PATH)
        PATHS.append(TEST_DIST_PATH)
        
        X_train = X_train[:, i].reshape(-1, 1)
        X_test = X_test[:, i].reshape(-1, 1)
        
        train_distance = TrainDistanceMixin(EuclideanDist())
        train_distance.precompute_distances(X_train, n_jobs=n_jobs)
        pickle.dump(train_distance, open(TRAIN_DIST_PATH, "wb"))

        test_distance = TestDistanceMixin(
            EuclideanDist(), selected_objects=np.arange(X_train.shape[0])
        )
        test_distance.precompute_distances(X_train, X_test=X_test, n_jobs=n_jobs)
        pickle.dump(test_distance, open(TEST_DIST_PATH, "wb"))

        Path(TRAIN_DIST_PATH).unlink()
        Path(TEST_DIST_PATH).unlink()
    return PATHS

In [ ]:
def dist_4_hist(X_train, X_test, column_name, base_path, n_jobs):
    PATHS = []
    path = base_path + column_name.replace(' ', '_') + '_' + 'wasserstein'
    TRAIN_DIST_PATH = path + '_tr.pickle'
    TEST_DIST_PATH = path + '_te.pickle'
    PATHS.append(TRAIN_DIST_PATH)
    PATHS.append(TEST_DIST_PATH)
    
    train_distance = TrainDistanceMixin(WassersteinDist())
    train_distance.precompute_distances(X_train, n_jobs=n_jobs)
    pickle.dump(train_distance, open(TRAIN_DIST_PATH, "wb"))

    test_distance = TestDistanceMixin(
        WassersteinDist(), selected_objects=np.arange(X_train.shape[0])
    )
    test_distance.precompute_distances(X_train, X_test=X_test, n_jobs=n_jobs)
    pickle.dump(test_distance, open(TEST_DIST_PATH, "wb"))

    Path(TRAIN_DIST_PATH).unlink()
    Path(TEST_DIST_PATH).unlink()
    return PATHS

In [ ]:
def dist_4_ts(X_train, X_test, column_name, base_path, n_jobs):
    PATHS = []
    path = base_path + column_name.replace(' ', '_') + '_' + 'ts_euc'
    TRAIN_DIST_PATH = path + '_tr.pickle'
    TEST_DIST_PATH = path + '_te.pickle'
    PATHS.append(TRAIN_DIST_PATH)
    PATHS.append(TEST_DIST_PATH)
    
    train_distance = TrainDistanceMixin(EuclideanDist())
    train_distance.precompute_distances(X_train, n_jobs=n_jobs)
    pickle.dump(train_distance, open(TRAIN_DIST_PATH, "wb"))

    test_distance = TestDistanceMixin(
        EuclideanDist(), selected_objects=np.arange(X_train.shape[0])
    )
    test_distance.precompute_distances(X_train, X_test=X_test, n_jobs=n_jobs)
    pickle.dump(test_distance, open(TEST_DIST_PATH, "wb"))

    Path(TRAIN_DIST_PATH).unlink()
    Path(TEST_DIST_PATH).unlink()
    return PATHS

In [54]:
BASE_PATH = '../data/bus/distances/'
n_jobs = 4

for column_name in X_train.columns:
    name = column_name.split('_')[1]
    PATHS = [] 
    if column_name.split('_')[0] == 'Numeric':
        # pass X_train[column_name].values, X_test[column_name].values and pass name
        PATHS.append(dist_4_numeric(X_train[column_name].values, X_test[column_name].values, name,BASE_PATH, n_jobs))
    if column_name.split('_')[0] == 'HistDens':
        PATHS.append(dist_4_hist(X_train[column_name].values, X_test[column_name].values, name,BASE_PATH, n_jobs))
    if column_name.split('_')[0] == 'TimeSeries':
        PATHS.append(dist_4_ts(X_train[column_name].values, X_test[column_name].values, name,BASE_PATH, n_jobs))

num
TSeries
num
TSeries
num
TSeries
num
TSeries
num
TSeries
num
TSeries
num
TSeries
num
TSeries
num
TSeries
num
TSeries
Hist
Hist
Hist
Hist
Hist
Hist
Hist
Hist
Hist
Hist


In [11]:
df = pd.read_parquet('CS_test.parquet')

In [12]:
sample_dist = TrainDistanceMixin(WassersteinDist())

In [13]:
sample_dist.precompute_distances(sample_hist[:1000])

In [12]:
from pathlib import Path
from risf.risf_data import RisfData
from risf.forest import RandomIsolationSimilarityForest

SAMPLE_X_train = X_train['HistDens_WetTankAirPressure'].values[:800]
SAMPLE_X_test = X_test['HistDens_WetTankAirPressure'].values[:200]

n_jobs = 4
TRAIN_DIST_PATH = '../data/bus/sample_test.pickle'
TEST_DIST_PATH = '../data/bus/sample_train.pickle'

train_distance = TrainDistanceMixin(WassersteinDist())
train_distance.precompute_distances(SAMPLE_X_train, n_jobs=n_jobs)
pickle.dump(train_distance, open(TRAIN_DIST_PATH, "wb"))

test_distance = TestDistanceMixin(
    WassersteinDist(), selected_objects=np.arange(SAMPLE_X_train.shape[0])
)
test_distance.precompute_distances(SAMPLE_X_train, X_test=SAMPLE_X_test, n_jobs=n_jobs)
pickle.dump(test_distance, open(TEST_DIST_PATH, "wb"))

X_risf = RisfData()
X_risf.add_data(
    np.arange(SAMPLE_X_train.shape[0]).reshape(-1, 1), dist=[TRAIN_DIST_PATH]
)  # I pass indices not full vectors

risf = RandomIsolationSimilarityForest(
    random_state=0, distances=X_risf.distances, n_jobs=n_jobs
).fit(X_risf)

SAMPLE_X_test_risf = X_risf.transform(
    [np.arange(SAMPLE_X_test.shape[0]).reshape(-1, 1)],
    forest=risf,
    precomputed_distances=[[TEST_DIST_PATH]],
    n_jobs=n_jobs,
)
print(risf.predict(SAMPLE_X_test_risf, return_raw_scores=False))

Path(TRAIN_DIST_PATH).unlink()
Path(TEST_DIST_PATH).unlink()



[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [25]:
first_elements = np.array([x[0] for x in df_369.iloc[:,0].values])

In [26]:
first_elements

array([ 6.42293568, 10.10835729, 11.02539229, ..., 10.93317167,
       10.91514789,  9.92134595])